In [1]:
import os
import torch, pyro, numpy as np
torch.set_default_tensor_type(torch.cuda.FloatTensor)
from torch import tensor
import torch.nn as nn
import torchvision.transforms.functional as TF
from tqdm import tqdm
import pandas as pd


from clipppy import load_config, Clipppy
from clipppy.patches import torch_numpy
from ruamel.yaml import YAML

import swyft
import pyro.distributions as dist


DEVICE = 'cuda'

from utils import *
from network import UNET, CustomHead

### Utilities

In [2]:
# @click.command()
# @click.option("--m",    type=int, default = 12,  help="Exponent of subhalo mass.")
# @click.option("--nsub", type=int, default = 1,   help="Number of subhaloes.")
# @click.option("--nsim", type=int, default = 100, help="Number of simulations to run.")

# @click.option("--lr",         type=float, default = 1e-3, help="Learning rate.")
# @click.option("--factor",     type=float, default = 1e-1, help = "Factor of Scheduler")
# @click.option("--patience",   type=int,   default = 5,    help = "Patience of Scheduler")
# @click.option("--max_epochs", type=int,   default = 30,   help = "Max number of epochs.")

In [2]:
# m = 10
# nsub = 1
# nsim = 10000

# lr = 1e-3
# factor = 1e-1
# patience = 5
# max_epochs = 1

In [3]:
SYSTEM_NAME = "ngc4414"

RUN = f'_m{m}_nsub{nsub}_nsim{nsim}'
SIM_PATH = f'/nfs/scratch/eliasd/store{RUN}.zarr' 

print(RUN)

assert os.path.exists(f'/nfs/scratch/eliasd/store{RUN}.sync')

_m12_nsub1_nsim10000


In [5]:
store = swyft.DirectoryStore(path=SIM_PATH)
print(f'Store has {len(store)} simulations')

Loading existing store.
Store has 9998 simulations


In [6]:
torch.set_default_tensor_type(torch.cuda.FloatTensor)  # HACK
CONFIG = get_config(SYSTEM_NAME, str(nsub), str(m))
torch.set_default_tensor_type(torch.FloatTensor)

In [7]:
prior, uv = get_prior(CONFIG)

### Train

In [8]:
idx = 0
img_0 = store[idx][0]['image']
L1, L2 = tensor(img_0.shape)
assert L1 == L2
L = L1.item()
print(f'L = {L}')

L = 40


In [9]:
torch.set_default_tensor_type(torch.FloatTensor)
dataset = swyft.Dataset(nsim, prior, store)#, simhook = noise)
marginals = [i for i in range(L**2)]
post = swyft.Posteriors(dataset)

In [5]:
save_name, save_path = get_name(RUN, lr, factor, patience)

In [11]:
print(f'Training {save_name}!')
    
torch.set_default_tensor_type(torch.FloatTensor)
post = swyft.Posteriors(dataset)
post.add(marginals, device = DEVICE, head = CustomHead, tail = UNET)
post.train(marginals, max_epochs = max_epochs,
           optimizer_args = dict(lr=lr),
           scheduler_args = dict(factor = factor, patience = patience)
          )

post.save(save_path)
print('Done!')

Training UNet_m12_nsub1_nsim10000_lr-3.0_fac-1.0_pat5.pt!
Training: lr=0.001, Epoch=5, VL=2217
Done!


In [12]:
save_path

'posts/UNet_m12_nsub1_nsim10000_lr-3.0_fac-1.0_pat5.pt'